In [1]:
from fairseq.models.bart import BARTModel

2021-10-06 18:39:03.099000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-06 18:39:03.099175: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# # MBart Test
# mbart = BARTModel.from_pretrained('mbart.cc25.v2', checkpoint_file='model.pt')
# mbart.eval()

In [13]:
plbart = BARTModel.from_pretrained('PLBART/plbart_orig_pretrained_ckpt', checkpoint_file='checkpoint_11_100000.pt')
plbart.eval()

BARTHubInterface(
  (model): BARTModel(
    (encoder): TransformerEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(50005, 768, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDropout()
     

In [34]:
fs_model = plbart.model

In [38]:
from transformers import PLBartConfig, PLBartForConditionalGeneration

In [39]:
hf_model = PLBartForConditionalGeneration.from_pretrained('plbart-base')

## Inputs

In [43]:
import sentencepiece as spm

In [44]:
vocab_filepath = "./PLBART/plbart_orig_pretrained_ckpt/sentencepiece.bpe.model"
tokenizer = spm.SentencePieceProcessor()
tokenizer.Load(vocab_filepath)
tokenizer.SetEncodeExtraOptions("")

True

In [55]:
texts = ["This is a sample text", "Another example here"]
text_0_tokens = tokenizer.EncodeAsIds(texts[0].strip())
text_1_tokens = tokenizer.EncodeAsIds(texts[1].strip())
# Need to check how is the original tokenizer

In [56]:
text_1_tokens += [1]*2

In [81]:
import numpy as np
import torch
import torch.nn as nn

In [66]:
input_ids = torch.from_numpy(np.array([text_0_tokens, text_1_tokens]))
attention_mask = torch.ones_like(input_ids)
attention_mask[1, -2:] = 0
token_type_ids = torch.zeros_like(input_ids)

# Encoder Embeddings

## Fairseq

In [84]:
fs_embeds, embed = fs_model.encoder.forward_embedding(input_ids)

In [85]:
# fs_model.encoder.quant_noise # None
# fs_model.encoder.layernorm_embedding #  not None
# fs_model.encoder.embed_positions # not None

## HuggingFace

In [86]:
hf_model.model.encoder.training

False

In [87]:
input_shape = input_ids.size()
inputs_embeds = hf_model.model.encoder.embed_tokens(input_ids) * hf_model.model.encoder.embed_scale
embed_pos = hf_model.model.encoder.embed_positions(input_shape)
hf_embeds = inputs_embeds + embed_pos
hf_embeds = hf_model.model.encoder.layernorm_embedding(hf_embeds)
hf_embeds = nn.functional.dropout(hf_embeds, p= hf_model.model.encoder.dropout, training= hf_model.model.encoder.training)


In [90]:
torch.allclose(hf_embeds[0], fs_embeds[0], atol=1e-5)

True

In [91]:
torch.allclose(hf_embeds[1,:-2], fs_embeds[1, :-2], atol=1e-5)

True

In [93]:
hf_embeds[1, -2:]

tensor([[-0.8719,  0.3062,  1.0597,  ..., -0.0096,  0.4988, -1.4163],
        [-0.8947,  0.2729,  0.8975,  ...,  0.3601,  0.6190, -1.4200]],
       grad_fn=<SliceBackward>)

In [94]:
fs_embeds[1, -2:] # Fairseq handles padding tokens differently, so they won't match.

tensor([[-0.6187,  0.1517,  1.1726,  ...,  0.1968,  0.5475, -1.0572],
        [-0.6187,  0.1517,  1.1726,  ...,  0.1968,  0.5475, -1.0572]],
       grad_fn=<SliceBackward>)

In [92]:
torch.allclose(hf_embeds[1,-2:], fs_embeds[1, -2:], atol=1e-5)

False

# Encoder Model

## Fairseq

In [67]:
fs_encoder = fs_model.encoder

In [69]:
fs_encoder_out = fs_encoder(input_ids, token_type_ids=token_type_ids)

TypeError: forward() got an unexpected keyword argument 'attention_mask'

## HuggingFace

In [68]:
hf_encoder = hf_model.model.encoder